In [4]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")  # Replace with the actual dataset name

# Check the structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4000
    })
})


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from datasets import load_dataset

# Load the Hugging Face dataset
dataset = load_dataset("pantelism/cats-vs-dogs")

# Convert to TensorFlow Dataset and resize images
def preprocess(example):
    image = tf.image.resize(example['image'], (150, 150)) / 255.0  # Rescale to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return image, label

train_data = dataset['train'].map(preprocess).with_options(tf.data.Options())
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Split training and validation
train_size = int(0.8 * len(dataset['train']))
validation_data = train_data.skip(train_size).take(len(dataset['train']) - train_size)
train_data = train_data.take(train_size)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_small.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, validation_data):
    # Get true labels and predictions
    y_true = np.concatenate([y for x, y in validation_data], axis=0)
    y_pred = model.predict(validation_data)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

evaluate_model(model, validation_data)


Map:   0%|                               | 0/4000 [00:00<?, ? examples/s]


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'tuple'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    # Resize and normalize the image
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}

# Apply preprocessing
train_data = dataset['train'].map(preprocess).with_options(tf.data.Options())
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data,
    epochs=50,
    validation_data=train_data,  # for illustration; ideally, use a separate validation set
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

# Evaluate the model
evaluate_model(model, train_data)


Map: 100%|████████████████████| 4000/4000 [09:59<00:00,  6.68 examples/s]


AttributeError: 'Dataset' object has no attribute 'with_options'

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    # Resize and normalize the image
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}

# Apply preprocessing
train_data = dataset['train'].map(preprocess)
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data,
    epochs=50,
    validation_data=train_data,  # for illustration; ideally, use a separate validation set
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

# Evaluate the model
evaluate_model(model, train_data)


TypeError: Dataset.shuffle() got an unexpected keyword argument 'buffer_size'

In [11]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}  # Return a dictionary

# Apply preprocessing to the training data
train_data = dataset['train'].map(preprocess).with_options(tf.data.Options())
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Split training and validation datasets
train_size = int(0.8 * len(train_data))
validation_data = train_data.skip(train_size)
train_data = train_data.take(train_size)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

evaluate_model(model, validation_data)


Map: 100%|████████████████████| 4000/4000 [08:10<00:00,  8.16 examples/s]


AttributeError: 'Dataset' object has no attribute 'with_options'

In [14]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}  # Return a dictionary

# Apply preprocessing to the training data
train_data = dataset['train'].map(preprocess)  # Removed with_options
train_data = train_data.shuffle(seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

# Split training and validation datasets
train_size = int(0.8 * len(train_data))
validation_data = train_data.skip(train_size)
train_data = train_data.take(train_size)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

evaluate_model(model, validation_data)


Batching examples: 100%|██████| 4000/4000 [05:05<00:00, 13.08 examples/s]


AttributeError: 'Dataset' object has no attribute 'prefetch'

In [19]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}  # Return a dictionary

# Apply preprocessing to the training data
train_data = dataset['train'].map(preprocess)  # Removed with_options

# Convert to TensorFlow Dataset
train_data_tf = train_data.to_tf_dataset(
    columns=['image'],  # Specify the input columns
    label_cols=['label'],  # Specify the label columns
    shuffle=True,
    batch_size=32,
)

# Split training and validation datasets
train_size = int(0.8 * len(train_data))
validation_data = train_data_tf.skip(train_size)
train_data_tf = train_data_tf.take(train_size)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data_tf,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

evaluate_model(model, validation_data)


ImportError: Called a Tensorflow-specific function but Tensorflow is not installed.

In [16]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [18]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))


tf.Tensor(-1000.1573, shape=(), dtype=float32)


In [20]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset from Hugging Face
dataset = load_dataset("pantelism/cats-vs-dogs")

# Preprocess function
def preprocess(example):
    image = tf.image.resize(example['image'], (150, 150))
    image = image / 255.0  # Normalize to [0, 1]
    label = tf.cast(example['label'], tf.int32)
    return {'image': image, 'label': label}  # Return a dictionary

# Apply preprocessing to the training data
train_data = dataset['train'].map(preprocess)  # Removed with_options

# Convert to TensorFlow Dataset
train_data_tf = train_data.to_tf_dataset(
    columns=['image'],  # Specify the input columns
    label_cols=['label'],  # Specify the label columns
    shuffle=True,
    batch_size=32,
    collate_fn=lambda x: {'image': tf.stack([item['image'] for item in x]),
                           'label': tf.stack([item['label'] for item in x])},
)


# Split training and validation datasets
train_size = int(0.8 * len(train_data))
validation_data = train_data_tf.skip(train_size)
train_data_tf = train_data_tf.take(train_size)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_data_tf,
    validation_data=validation_data,
    epochs=50,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_model.h5')

# Plotting training history
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_training_history(history)

# Confusion Matrix and ROC Curve
def evaluate_model(model, data):
    # Get true labels and predictions
    y_true = np.concatenate([y['label'].numpy() for y in data], axis=0)
    y_pred = np.concatenate([model.predict(y['image']) for y in data], axis=0)
    y_pred_classes = (y_pred > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC Curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

evaluate_model(model, validation_data)


ImportError: Called a Tensorflow-specific function but Tensorflow is not installed.

In [21]:
# Cats vs. Dogs Classification with Explainability

In this notebook, we implement two popular explainability methods for convolutional neural networks (CNNs): **Integrated Gradients** and **Grad-CAM**. These methods help us understand how the model makes its predictions by highlighting the important regions in the input images.

We will use the trained model from our previous work on the Cats vs. Dogs classification task. The model achieved a certain level of accuracy using data augmentation, and now we will explain its decisions.

## Table of Contents
1. [Setup](#setup)
2. [Load the Model](#load-the-model)
3. [Integrated Gradients](#integrated-gradients)
4. [Grad-CAM](#grad-cam)
5. [Results and Conclusion](#results-and-conclusion)


SyntaxError: invalid syntax (4268408557.py, line 3)

In [28]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import cv2
from captum.attr import IntegratedGradients
from captum.attr import GradCam
from captum.attr import visualize_image_attr

# Load your trained model
model = torch.load('cats_and_dogs_model.pth')  # Load your saved model
model.eval()

# Preprocessing function for images
def preprocess(image):
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Function to visualize Integrated Gradients
def visualize_integrated_gradients(image_tensor):
    ig = IntegratedGradients(model)
    # Make sure the model is in evaluation mode
    model.eval()
    
    # Preprocess the image
    image_tensor = preprocess(image_tensor)

    # Compute attributions
    attr_ig = ig.attribute(image_tensor, target=1)  # Assuming '1' is the target class

    # Visualize
    attr_ig_np = attr_ig.squeeze().detach().numpy().transpose(1, 2, 0)  # Convert to HWC
    visualize_image_attr(attr_ig_np, method='heat_map', sign='all', title='Integrated Gradients Attribution')
    plt.axis('off')
    plt.show()

# Function to visualize Grad-CAM
def visualize_gradcam(image_tensor):
    grad_cam = GradCam(model, model.conv2)  # Adjust layer as per your model architecture
    model.eval()
    
    # Preprocess the image
    image_tensor = preprocess(image_tensor)

    # Compute Grad-CAM attributions
    cam = grad_cam.attribute(image_tensor, target=1)

    # Rescale to [0, 1]
    cam_np = cam.squeeze().detach().numpy()
    cam_np = np.maximum(cam_np, 0)
    cam_np /= cam_np.max()  # Normalize

    # Convert to heatmap
    heatmap = cv2.resize(cam_np, (150, 150))  # Resize to original image size
    heatmap = np.uint8(255 * heatmap)  # Convert to 8-bit image
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # Apply color map

    # Overlay heatmap on original image
    original_image_np = image_tensor.squeeze().permute(1, 2, 0).numpy()  # Convert to HWC
    overlay = cv2.addWeighted(original_image_np, 0.5, heatmap, 0.5, 0)

    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Grad-CAM')
    plt.show()

# Example usage
# Load your image using PIL or any other method
from PIL import Image
image = Image.open("path_to_your_image.jpg")

visualize_integrated_gradients(image)
visualize_gradcam(image)


ImportError: cannot import name 'GradCam' from 'captum.attr' (/Users/vi.rubio/eng-ai-agents/venv/lib/python3.11/site-packages/captum/attr/__init__.py)

In [29]:
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from captum.attr import IntegratedGradients
from captum.attr import LayerGradCam
from captum.attr import visualize_image_attr

# Load your trained model
model = torch.load('cats_and_dogs_model.pth')  # Load your saved model
model.eval()

# Preprocessing function for images
def preprocess(image):
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Function to visualize Integrated Gradients
def visualize_integrated_gradients(image_tensor):
    ig = IntegratedGradients(model)
    model.eval()
    
    image_tensor = preprocess(image_tensor)

    attr_ig = ig.attribute(image_tensor, target=1)

    attr_ig_np = attr_ig.squeeze().detach().numpy().transpose(1, 2, 0)
    visualize_image_attr(attr_ig_np, method='heat_map', sign='all', title='Integrated Gradients Attribution')
    plt.axis('off')
    plt.show()

# Function to visualize Grad-CAM
def visualize_gradcam(image_tensor):
    # Use LayerGradCam for Grad-CAM
    layer_grad_cam = LayerGradCam(model, model.conv2)  # Adjust layer as per your model architecture
    model.eval()
    
    image_tensor = preprocess(image_tensor)

    cam = layer_grad_cam.attribute(image_tensor, target=1)

    cam_np = cam.squeeze().detach().numpy()
    cam_np = np.maximum(cam_np, 0)
    cam_np /= cam_np.max()

    heatmap = plt.cm.jet(cam_np)[:, :, :3]  # Create a heatmap using Matplotlib

    original_image_np = image_tensor.squeeze().permute(1, 2, 0).numpy()
    overlay = (heatmap * 0.5 + original_image_np * 0.5)

    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Grad-CAM')
    plt.show()

# Example usage
from PIL import Image
image = Image.open("path_to_your_image.jpg")

visualize_integrated_gradients(image)
visualize_gradcam(image)


ImportError: cannot import name 'visualize_image_attr' from 'captum.attr' (/Users/vi.rubio/eng-ai-agents/venv/lib/python3.11/site-packages/captum/attr/__init__.py)

In [30]:
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from captum.attr import IntegratedGradients
from captum.attr import LayerGradCam
from captum.attr import visualization as viz  # Updated import for visualization

# Load your trained model
model = torch.load('cats_and_dogs_model.pth')  # Load your saved model
model.eval()

# Preprocessing function for images
def preprocess(image):
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Function to visualize Integrated Gradients
def visualize_integrated_gradients(image_tensor):
    ig = IntegratedGradients(model)
    model.eval()
    
    image_tensor = preprocess(image_tensor)

    attr_ig = ig.attribute(image_tensor, target=1)

    attr_ig_np = attr_ig.squeeze().detach().numpy().transpose(1, 2, 0)

    # Visualize the Integrated Gradients attribution
    viz.visualize_image_attr(attr_ig_np, method='heat_map', sign='all', title='Integrated Gradients Attribution')
    plt.axis('off')
    plt.show()

# Function to visualize Grad-CAM
def visualize_gradcam(image_tensor):
    # Use LayerGradCam for Grad-CAM
    layer_grad_cam = LayerGradCam(model, model.conv2)  # Adjust layer as per your model architecture
    model.eval()
    
    image_tensor = preprocess(image_tensor)

    cam = layer_grad_cam.attribute(image_tensor, target=1)

    cam_np = cam.squeeze().detach().numpy()
    cam_np = np.maximum(cam_np, 0)
    cam_np /= cam_np.max()

    heatmap = plt.cm.jet(cam_np)[:, :, :3]  # Create a heatmap using Matplotlib

    original_image_np = image_tensor.squeeze().permute(1, 2, 0).numpy()
    overlay = (heatmap * 0.5 + original_image_np * 0.5)

    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Grad-CAM')
    plt.show()

# Example usage
from PIL import Image
image = Image.open("path_to_your_image.jpg")

visualize_integrated_gradients(image)
visualize_gradcam(image)


/var/folders/qs/5tb486nj6bv4jc33tdzjs8qr0000gn/T/ipykernel_34644/938220871.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('cats_and_dogs_model.pth')

FileNotFoundError: [Errno 2] No such file or directory: 'cats_and_dogs_model.pth'